In [ ]:
import pandas as pd
from IPython.display import display

df = pd.read_csv("../data/metazoa_nodes.csv", dtype={"ott_id": str, "parent_ott_id": str})

print("Total rows:", len(df))
display(df.head())

# Named nodes missing OTT IDs
named = df[df['name'].notnull() & (df['name'] != '')]
missing_ott = named[named['ott_id'].isna()]
print("Named nodes without ott_id:", len(missing_ott))
if len(missing_ott) > 0:
    display(missing_ott.sample(min(5, len(missing_ott))))


# Non-root nodes missing parent IDs
nonroot = df[df['ott_id'].notna() & (df['ott_id'] != "691846")]
missing_parent = nonroot[nonroot['parent_ott_id'].isna()]
print("Non-root nodes without parent_ott_id:", len(missing_parent))
if len(missing_parent) > 0:
    display(missing_parent.sample(min(5, len(missing_parent))))


# Invalid parent IDs
valid_ids = set(df['ott_id'].dropna())
bad_refs = df[df['parent_ott_id'].notna() & ~df['parent_ott_id'].isin(valid_ids)]
print("Invalid parent_ott_id references:", len(bad_refs))
if len(bad_refs) > 0:
    display(bad_refs.sample(min(5, len(bad_refs))))


print("\nTaxonomic rank distribution:")
df['rank_depth'] = df['name'].str.count(' ')
depth_counts = df['rank_depth'].value_counts().sort_index()
display(depth_counts)


Total rows: 2870658


,ott_id,name,parent_ott_id
0,691846,Metazoa,NaN
1,67819,Porifera,691846
2,4939695,Clathrinidae environmental samples,67819
3,5260763,unclassified Clathrinidae,67819
4,5261574,Clathrinidae sp. BC12,67819


Named nodes without ott_id: 0
Non-root nodes without parent_ott_id: 0
Invalid parent_ott_id references: 0

Taxonomic rank distribution:


rank_depth
0      259177
1     1907433
2      603566
3       69287
4       23795
5        5478
6        1419
7         400
8          62
9          19
10         11
11          4
12          6
13          1
Name: count, dtype: int64